# Finding the Churned Customers to Target for Promotions

Online store of company has $1000 left in marketing budget and want to use that to send coupons to customers. Manager has decided to target the customers of physical store. But to avoid stealing the customers of physical store, he decided to only target churned customers. 

As a data analysts, we have a list of the best physical store customers (name and address) to send coupons to. We should use my best judgment to figure out what the value of the coupons should be and to how many people we can send them to. We'll figure what criteria to use to determine who the "best" customers are. 

So in this project, we've to 
* Define churned customers
* Find out the value for coupon
* Decide how many customers to send coupons to.


# Data Exploring 

[This](https://github.com/prab-hub/coupons/blob/main/Retail_Data_Transactions.csv) is the dataset we will be analysing. Let's explore it as Pandas DataFrame.

In [1]:
import pandas as pd
import datetime as dt
data = pd.read_csv("C:/Users/Linus/Documents/Sheets/Retail_Data_Transactions.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  125000 non-null  object
 1   trans_date   125000 non-null  object
 2   tran_amount  125000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.9+ MB


As we can see above, there are 125K transaction entries. Also data only contains customer ID, transaction dates & amounts. SO we just have to find transaction ID of best churned customers.

# Churned Customers

We will group the transactions by `customer_id` and keep last transaction in `trans_date` column.

In [2]:
group_by_customer = data.groupby("customer_id")
last_transaction = group_by_customer['trans_date'].max()
best_churn = pd.DataFrame(last_transaction)
best_churn

,trans_date
customer_id,
CS1112,29-Apr-14
CS1113,30-Mar-12
CS1114,31-Dec-14
CS1115,27-Jun-13
CS1116,30-May-14
...,...
CS8996,29-Sep-12
CS8997,31-May-12
CS8998,27-Jul-12


We are left with ~7000 customers. 

Since we don't have concrete definition of churn, we will use Sept 16th, 2019 (3 months before analysis date) as a cut-off date. Anyone who hasn't purchased after cut-off date is churned customer. 

We will also calculate `nr_of_transactions` and `amount_spent` for each churned customer.

In [3]:
best_churn['trans_date'] = pd.to_datetime(best_churn['trans_date'])
cutoff_day = dt.datetime(2019, 9, 16)
best_churn["churned"] = best_churn["trans_date"].apply(lambda date:1 if date < cutoff_day else 0)
best_churn["churned"].value_counts()

1    6889
Name: churned, dtype: int64

In [4]:
best_churn["nr_of_transactions"] = group_by_customer.size()
best_churn["amount_spent"] = group_by_customer.sum()
best_churn.drop("trans_date", axis="columns", inplace=True)

C:\Users\Linus\AppData\Local\Temp\ipykernel_7096\2235534557.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  best_churn["amount_spent"] = group_by_customer.sum()


In [11]:
best_churn

,churned,nr_of_transactions,amount_spent
customer_id,,,
CS1112,1,15,1012
CS1113,1,20,1490
CS1114,1,19,1432
CS1115,1,22,1659
CS1116,1,13,857
...,...,...,...
CS8996,1,13,582
CS8997,1,14,543
CS8998,1,13,624


In [13]:
best_churn[["nr_of_transactions", "amount_spent"]].describe()

,nr_of_transactions,amount_spent
count,6889.000000,6889.000000
mean,18.144869,1179.269705
std,5.193014,465.832609
min,4.000000,149.000000
25%,14.000000,781.000000
50%,18.000000,1227.000000
75%,22.000000,1520.000000
max,39.000000,2933.000000


# Ranking Customers

We'll now focus on finding the best customers. This is a two-part problem: Find a ranking mechanism. Determine a threshold to identify the best customers.

We will use a very simple weighted sum model to classify customers. Think of it as a scoring model that assigns a number to each customer that represents how good a customer they are.

In this model, we will take two criteria into account: Amount spent and number of purchases made, and that the scores should be the same weight. This means that a customer who spent a lot is worth as much as someone who made many purchases.

More formally, you define the score of a customer as:

(1/2 * `Number of purchases`) + (1/2 * `Amount spent`)

But if a customer has made two purchases totalling $500, their score would be 251. If a customer has spent $400 across 20 different purchases, their score would be 210.

The second customer is clearly a regular customer and shows potential to spend more than the first in the long run, but it is scored lower than the first. This score with the data as is, favours money spent over the number of purchases. 

This happens for two reasons:

+ Money spent is usually much higher than the number of transactions;
+ We are using the same weight (1/2) for both criteria

So we will use the min-max scaling technique to rescale both columns so that we can make an apples-to-apples comparison.

Mathematically, we can do this by using the formula 

`(X-min(X))/(max(X)-min(X))`

where `X` represents the column we want to scale. 

In [21]:
best_churn["scaled_tran"] = (best_churn["nr_of_transactions"] - \
best_churn["nr_of_transactions"].min()) / (best_churn["nr_of_transactions"].max() - \
best_churn["nr_of_transactions"].min())

best_churn["scaled_tran"]

customer_id
CS4424    1.000000
CS4320    0.971429
CS5752    0.828571
CS4660    0.828571
CS3799    0.914286
            ...   
CS8841    0.028571
CS8077    0.028571
CS8234    0.028571
CS8559    0.000000
CS7333    0.000000
Name: scaled_tran, Length: 6889, dtype: float64

In [22]:
best_churn["scaled_amount"] = (best_churn["amount_spent"] - \
best_churn["amount_spent"].min()) / (best_churn["amount_spent"].max() - \
best_churn["amount_spent"].min())

best_churn["scaled_amount"]

customer_id
CS4424    1.000000
CS4320    0.897270
CS5752    0.884698
CS4660    0.854167
CS3799    0.849138
            ...   
CS8841    0.011494
CS8077    0.005388
CS8234    0.002874
CS8559    0.002874
CS7333    0.000000
Name: scaled_amount, Length: 6889, dtype: float64

In [23]:
best_churn['score'] = 100* (0.5*best_churn["scaled_amount"] + 0.5*best_churn["scaled_tran"])
best_churn['score']

customer_id
CS4424    100.000000
CS4320     93.434934
CS5752     85.663485
CS4660     84.136905
CS3799     88.171182
             ...    
CS8841      2.003284
CS8077      1.697968
CS8234      1.572250
CS8559      0.143678
CS7333      0.000000
Name: score, Length: 6889, dtype: float64

In [24]:
best_churn.sort_values("score", inplace=True, ascending=False)
best_churn

,churned,nr_of_transactions,amount_spent,scaled_tran,scaled_amount,score
customer_id,,,,,,
CS4424,1,39,2933,1.000000,1.000000,100.000000
CS4320,1,38,2647,0.971429,0.897270,93.434934
CS3799,1,36,2513,0.914286,0.849138,88.171182
CS5109,1,35,2506,0.885714,0.846624,86.616892
CS3805,1,35,2453,0.885714,0.827586,85.665025
...,...,...,...,...,...,...
CS7716,1,4,221,0.000000,0.025862,1.293103
CS7224,1,4,191,0.000000,0.015086,0.754310
CS8504,1,4,190,0.000000,0.014727,0.736351


# Determining a Threshold

Now that we have a way to compare customers, we need to decide on a threshold to determine which customers are "the best." Should it be the first 20 customers? The first 40 customers? The top 10% of customers? What are the criteria?

Here are some factors that we decided to take into account:

* The budget is $1,000.
* No indication was given about how much each coupon would be worth — it's for us to decide.
* The coupons need to be good enough to prompt people to actually use them.
* They can't be too high because:
    * That reduces the number of customers who get them.
    * It would be like giving away money.
    * Due to price dumping, it could be illegal.

From our experience, we know that a `30 %` discount is already very enticing;

With all this in mind, we decide to employ the following strategy to determine the cutoff point:

 * Find the mean of the transactions and compute 30% of that. Make this the value of the coupon;
* Divide the budget by the value obtained above to get the number of coupons we're going to be sending out;
* Pick the first `n` churned customers where `n` is the result of the calculation done in the previous step. This is your cutoff point.

In [27]:
mean_trans = data["tran_amount"].mean()
coupon = 0.30 * mean_trans
print("Coupan:", coupon)

nr_of_customers = 1000/coupon
print("No of customers:", nr_of_customers)

Coupan: 19.4975736
No of customers: 51.28843314123969


# Delivering the Results

Since we can't have 51 and a bit customers and a value of roughly `$19.50` is a weird value for a coupon, so we will round it to `$20`, and then decide to send the coupon to the top `50` churned customers.

We will save a dataframe as a text file.

In [28]:
top_50_churned = best_churn.loc[best_churn["churned"]==1].head(50)
top_50_churned.to_csv("best_customers.txt")

[This](https://github.com/prab-hub/coupons/blob/main/best_customers.txt) is our final list of 50 best customers. 